# Assignment No 3d
###### *Hassan Raza*
----
## Goal

Your goal in this part of assigment is to implement a Decision Tree Classifier for categorical variables.

**Note** Please note that you are allowed to use only those libraries which we have discussed in the class, i.e. numpy, scipy, pandas.

## Submission Instructions
You are required to submit the original notebook file on the google classroom (with .ipynb extension), with complete set of outputs. Students failing to do so will get zero marks. 

*Please read each step carefully and understand it fully before proceeding with code writing*

## Plagiarism
Any form of plagiarism will not be tolerated and result in 0 marks.



### Decision Tree Classifier

Now in this assignment we will be implementing the Decision Classifier for both Continuous and Categorical attributes.

Decision tree can be built by using any of the following split criterias, namely:
 - Information Gain
 - Gini Index
 - CART 

However, you are required here to implement the decision tree with information gain as splitting criterion.

Remember in my code i am not looking for maximizing the information gain, instead i am looking for minimizing the split entropy. Recall,
$$Information Gain  = H(D) - H(D_Y,D_N)$$

Where,

$H(D)$ is the data set entroy and $H(D_Y,D_N)$ is split entropy. Since $H(D)$ is constant for the given dataset so maximizing the entropy is equal to minimizing the split entropy and that is what is being represented in my code outputs.

In [1]:
%pylab inline
import numpy
import scipy.stats
from collections import defaultdict  # default dictionary 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
#Cutomize the Matplotlib for beautiful plots...
# import dmStyle
# dmStyle.customize_mpl()

In [3]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

In [4]:
def getSplits(categories):
    '''
        function returns list of splits for the given list of categorical variables...
        
        Input:
        ------------
            categories: a list of unique categories...
        
        Return:
        ------------
            list of splits(tuples) for given list of categorical variables. Each pair of sublists
            defines the left and right splits, e.g. This list
            [('y', 'f'), ('s', 'g'), ('f'), ('y', 's', 'g')]
            
            defines two splits with each pair representing a different split.
        Examples:
        ------------
        splits=getSplits(['a1','a2','a3','a4']) will return 
        [('a1', 'a2', 'a4'), ('a3',), 
        ('a2', 'a4'),('a1', 'a3'), 
        ('a1', 'a4'), ('a3', 'a2'), 
        ('a1', 'a3', 'a2'), ('a4',), 
        ('a3', 'a4'), ('a1', 'a2'), 
        ('a1',), ('a3', 'a2', 'a4'), 
        ('a2',), ('a1', 'a3', 'a4')] 
    '''

    categories = set(categories)
    tsplits = t.get_powerset(categories,len(categories)-1) # get all the power sets with the given cardinality...
    flist=[]
    for s in tsplits:
        if not s in flist:
            r = categories.difference(s)
            flist.append(s)
            flist.append(r)

    olist=[]
    for s in flist[::-1]:
        ilist=[]
        for k in s:
            ilist.append(k)
        olist.append(tuple(ilist))    
    return olist

In [5]:
splits = getSplits(['a1','a2','a3','a4'])
print(splits,len(splits))

[('a1', 'a2'), ('a3', 'a4'), ('a1', 'a3', 'a4'), ('a2',), ('a2', 'a3'), ('a1', 'a4'), ('a2', 'a4'), ('a1', 'a3'), ('a3',), ('a1', 'a2', 'a4'), ('a1',), ('a2', 'a3', 'a4'), ('a4',), ('a1', 'a2', 'a3')] 14


In [6]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=None,fidx=-1):
        '''
            purity: purity level at which to stop
            klasslabel: klasslabel of the node, (for leaf node)
            score: information gain of the newly added node
            split: splitting threshold
            fidx: feature index            
        '''
        self.lchild=None
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity
        self.ftype= 'categorical' if type(self.split) in [tuple, str, numpy.string_] else 'continuous'

    def set_childs(self,lchild,rchild):
        self.lchild = lchild
        self.rchild = rchild

    def isleaf(self):
        # YOUR CODE HERE
        if self.klasslabel == '':
            return False
        else:
            return True

    def isless_than_eq(self, X):
        # YOUR CODE HERE
        if self.ftype == 'categorical':
            if X[self.fidx] in self.split:
                return True
            else:
                return False
        else:
            if X[self.fidx] < self.split:
                return True
            else:
                return False

    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)
    

In [7]:
class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
    def __init(self):
        self.tree = None   
        self.purity = 0.0
        self.exthreshold = 0
        self.maxdepth = 0

    def __init__(self, purityp, exthreshold, maxdepth=10):        
        self.purity = purityp
        self.exthreshold = exthreshold
        self.maxdepth = maxdepth

    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
        '''
        nexamples, nfeatures = X.shape
        # # now go and train a model for each class...
        # YOUR CODE HERE
        self.tree = self.build_tree(X,Y,self.maxdepth)
        
    def build_tree(self, X, Y, depth):
        """ 
            Function is used to recursively build the decision Tree 
          
            Input
            -----
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns
            -------
            root node of the built tree...
        """
        nexamples, nfeatures = X.shape
        klasses = np.unique(Y)

        MostOccuringClassInY = pd.value_counts(Y).index[0]
        
        #Purity Of CurrentData
        Purity = pd.value_counts(Y).max() / len(Y)

        # YOUR CODE HERE
        if ((len(klasses) == 1) or (nexamples < self.exthreshold) or (Purity >= self.purity) or (depth < 0)):
            node = Node(Purity, MostOccuringClassInY)
        else:
            BestSplitPoint,BestSplitEntropy,BestLeftIndices,BestRightIndices,BestFeatureIndex = None,None,None,None,None
            MinimumSplitEntropy,SplitPoint,LeftDataIndices,RightDataIndices = None,None,None,None
            Threshold = float("inf")
            
            for FeatureIndex in range(nfeatures):
                if type(X[0][FeatureIndex]) in [tuple, str, numpy.string_]:
                    if len(np.unique(X[:,FeatureIndex])) == 1:
                        continue
                    else:
                        SplitPoint, MinimumSplitEntropy, LeftDataIndices, RightDataIndices = self.evaluate_categorical_attribute(deepcopy(X[:,FeatureIndex]), deepcopy(Y))
                else:
                    SplitPoint, MinimumSplitEntropy, LeftDataIndices, RightDataIndices = self.evaluate_numerical_attribute(deepcopy(X[:, FeatureIndex]), deepcopy(Y))
                
                if  MinimumSplitEntropy != None and MinimumSplitEntropy < Threshold:
                    BestFeatureIndex = FeatureIndex
                    BestSplitPoint = SplitPoint
                    BestSplitEntropy = MinimumSplitEntropy
                    BestLeftIndices = LeftDataIndices
                    BestRightIndices = RightDataIndices
                    Threshold = BestSplitEntropy
                    
            node = None
            if MinimumSplitEntropy == None:
                node = Node(Purity, MostOccuringClassInY)
            else:
                node = Node(Purity, '', BestSplitEntropy, BestSplitPoint, BestFeatureIndex)
                node.set_childs(self.build_tree(deepcopy(X[BestLeftIndices]), deepcopy(Y[BestLeftIndices]), depth - 1), self.build_tree(deepcopy(X[BestRightIndices]), deepcopy(Y[BestRightIndices]), depth - 1))
        return node
        
    def test(self, X):
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        
        nexamples, nfeatures = X.shape
        pclasses = self.predict(X)
        return np.array(pclasses)

    def evaluate_categorical_attribute(self, feat, Y):
            """
            Evaluates the cateogrical attribute for all possible split points, i.e. 2^(m-1)-2 for
            possible feature selection

            Input:
            ---------
            feat: a categorical feature
            Y: labels

            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node

            """
            categories = set(feat)
            classess, counts = np.unique(Y, return_counts=True)

            splits = getSplits(categories)

            VerySmallAbsoluteValue = 0.0000001
            total = np.sum(counts)
            total += VerySmallAbsoluteValue
            
            mingain,csplit,Xlidx,Xridx = None,None,None,None
            MinimumThreshold = float("inf")
            EntropyIndex = 0
            for LeftSplit in range(0,len(splits),2):
                RightSplit = LeftSplit + 1
    
                LeftData = feat[np.isin(feat, splits[LeftSplit])]
                RightData = feat[np.isin(feat, splits[RightSplit])]

                LeftY = Y[np.isin(feat, splits[LeftSplit])]
                RightY = Y[np.isin(feat, splits[RightSplit])]

                LeftDataClasses, LeftDataCounts = np.unique(LeftY, return_counts=True)
                RightDataClasses, RightDataCounts = np.unique(RightY, return_counts=True)

                LeftDataCounts = np.array(LeftDataCounts, dtype=float)
                RightDataCounts = np.array(RightDataCounts, dtype=float)
                LeftDataCounts += VerySmallAbsoluteValue
                RightDataCounts += VerySmallAbsoluteValue
                
                PartA = -1 * (np.sum(LeftDataCounts) / total) * np.sum((LeftDataCounts / np.sum(LeftDataCounts)) * (np.log2(LeftDataCounts / np.sum(LeftDataCounts))))
                PartB = -1 * (np.sum(RightDataCounts) / total) * np.sum((RightDataCounts / np.sum(RightDataCounts)) * (np.log2(RightDataCounts / np.sum(RightDataCounts))))
                SplitEntropy = PartA + PartB

                if SplitEntropy < MinimumThreshold:
                    MinimumThreshold = SplitEntropy
                    mingain = SplitEntropy
                    csplit = splits[LeftSplit]
                    Xlidx = np.where(np.isin(feat, splits[LeftSplit]))[0]
                    Xridx = np.where(np.isin(feat, splits[RightSplit]))[0]

            return csplit, mingain, Xlidx, Xridx
        
    def evaluate_numerical_attribute(self, feat, Y):
        '''
            Evaluates the numerical attribute for all possible split points for
            possible feature selection
            
            Input:
            ---------
            feat: a contiuous feature
            Y: labels
            
            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node
            
        '''
        
        classes,counts = np.unique(Y, return_counts=True)
        nclasses = len(classes)
        sidx = np.argsort(feat)
        f = feat[sidx] # sorted features
        sY = Y[sidx] # sorted features class labels...
                
        # YOUR CODE HERE
        #Finding all split points for the feature array
        UniquesInF = np.unique(sorted(f))
        SplitPoints = []
        for i in range(len(UniquesInF)-1):
            SplitPoints.append((UniquesInF[i] + UniquesInF[i+1])/2)
        '''
        After dry running this process, I found that I need to
        append a value greater than the maximum value of the feature array 
        so that the last largest value in feature array is also accomodated in 
        left and right data tables
        '''
        SplitPoints.append(np.max(f) + 1) 
        
        # Constructing the table using given data
        RightDataTable,LeftDataTable = np.zeros((len(SplitPoints)-1,nclasses)),np.zeros((len(SplitPoints)-1,nclasses))
        CurrentSplitPointIndex = 0
        TempVector = np.zeros(nclasses)
        for i in range(len(f)):
            if(f[i] < SplitPoints[CurrentSplitPointIndex]):
                TempVector[classes == sY[i]] += 1
            else:
                LeftDataTable[CurrentSplitPointIndex] = TempVector
                CurrentSplitPointIndex += 1
                TempVector[classes == sY[i]] += 1
        RightDataTable = TempVector - LeftDataTable

        #Adding a very small value in the table to avoid any 0s during entropy calculation
        VerySmallAbsoluteValue = 0.0000001
        LeftDataTable += VerySmallAbsoluteValue
        RightDataTable += VerySmallAbsoluteValue

        #Calculating Entropies using tables
        SplitEntropies = np.zeros(RightDataTable.shape[0])
        LeftSum = np.sum(LeftDataTable,axis=1)
        RightSum = np.sum(RightDataTable,axis=1)
        TotalSize = np.sum(counts)
        for i in range(RightDataTable.shape[0]):
            PartA = -1 * (LeftSum[i]/TotalSize) * np.sum((LeftDataTable[i]/LeftSum[i]) * np.log2(LeftDataTable[i]/LeftSum[i]))
            PartB = -1 * (RightSum[i]/TotalSize) * np.sum((RightDataTable[i]/RightSum[i]) * np.log2(RightDataTable[i]/RightSum[i]))
            SplitEntropies[i] = PartA + PartB
        
        #Required splitpoint,minimum split entropy, and left, right data at best split point
        split = SplitPoints[np.argmin(SplitEntropies)]
        mingain = np.min(SplitEntropies)
        Xlidx = np.where(feat < split)[0]
        Xridx = np.where(feat > split)[0]
        return split,mingain,Xlidx,Xridx

    def predict(self, X):
        """
        Test the trained classifiers on the given example X
        
                   
            Input:
            ------
            X: [1 x d] a d-dimensional test example.
           
            Returns:
            -----------
                pclass: the predicted class for the given example, i.e. to which it belongs
        """
        z = []
        for idx in range(X.shape[0]):
            z.append(self._predict(self.tree, X[idx, :]))
        return z 
    
    def _predict(self, node, X):
        # YOUR CODE HERE
        TreeRootTemp = node
        while TreeRootTemp.isleaf() == False: #Untill we reach a leaf node
            if TreeRootTemp.isless_than_eq(X) == True: #If the test example is less than the split point
                TreeRootTemp = TreeRootTemp.lchild
            else: #If the test example is greater than the split point
                TreeRootTemp = TreeRootTemp.rchild
        return TreeRootTemp.klasslabel #Return the class label of the test example  

    def __str__(self):
        str = '---------------------------------------------------'
        str += '\n A Decision Tree With Depth={}'.format(self.find_depth())
        str += self.__print(self.tree)
        str += '\n---------------------------------------------------'
        return str  # self.__print(self.tree)        
   
    def find_depth(self):
        return self._find_depth(self.tree)

    def _find_depth(self, node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild), self._find_depth(node.rchild)) + 1

    def __print(self, node, depth=0):
        ret = ""
        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild, depth + 1)
        # Print own value
        ret += "\n" + ("    "*depth) + node.get_str()
        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild, depth + 1)
        return ret

# Lets test our code for the given example in the book.

In [8]:
#load the Iris dataset
tdata=pd.read_csv('./iris.data')
tdata.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']

In [9]:
tx=tdata['SepalLength'].dropna()
tx[(tdata['SepalLength']>=4.3) & (tdata['SepalLength']<=5.2)]='a1'
tx[(tdata['SepalLength']>5.2) & (tdata['SepalLength']<=6.1)]='a2'
tx[(tdata['SepalLength']>6.1) & (tdata['SepalLength']<=7.0)]='a3'
tx[(tdata['SepalLength']>7.0) & (tdata['SepalLength']<=7.9)]='a4'

In [10]:
Y = tdata['Class'].dropna()
Y[Y=='Iris-virginica'] = 'Iris-versicolor'
Y = Y.values

In [11]:
dt = DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)
print(split, gain)

('a1',) 0.5107023543360183


In [12]:
from nose.tools import assert_almost_equal, assert_almost_equals
dt=DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)

assert_almost_equal('a1', split[0])
assert_almost_equal(gain, 0.51, 1)

# Now lets test on [Mushroom](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names) dataset


This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525).  Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended.  This latter class was combined with the poisonous one.  The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like ``leaflets three, let it be'' for Poisonous Oak and Ivy.


- Number of Instances: 8124

- Number of Attributes: 22 (all nominally valued)

- Attribute Information: (classes: edible=e, poisonous=p)
         1. cap-shape:                bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
         2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
         3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
         4. bruises?:                 bruises=t,no=f
         5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
         6. gill-attachment:          attached=a,descending=d,free=f,notched=n
         7. gill-spacing:             close=c,crowded=w,distant=d
         8. gill-size:                broad=b,narrow=n
         9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
        10. stalk-shape:              enlarging=e,tapering=t
        11. stalk-root:               bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
        12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
        13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
        14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
        15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
        16. veil-type:                partial=p,universal=u
        17. veil-color:               brown=n,orange=o,white=w,yellow=y
        18. ring-number:              none=n,one=o,two=t
        19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
        20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
        21. population:               abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
        22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d

- Missing Attribute Values: 2480 of them (denoted by "?"), all for  attribute #11.

- Class Distribution: 
        --    edible: 4208 (51.8%)
        -- poisonous: 3916 (48.2%)
        --     total: 8124 instances

In [13]:
#load the data set
data = pd.read_csv('./mushrooms.csv')
data.columns=columns=['class', 'cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
data

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [14]:
# Get your data in matrix
X = np.asarray(data.drop('class', axis=1))
Y = np.asarray(data.iloc[:,0].dropna())
print("Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape )  

Data Set Dimensions= (8124, 22)  True Class labels dimensions (8124,)


In [15]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest = t.split_data(X,Y)
print("Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape)  
print("Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape) 


Training Data Set Dimensions= (5687, 22) Training True Class labels dimensions (5687,)
Test Data Set Dimensions= (2437, 22) Test True Class labels dimensions (5687,)


In [16]:
# Lets train a Decision Tree Classifier two features
feat = np.arange(2)
dt = DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)

In [17]:
print(dt)

---------------------------------------------------
 A Decision Tree With Depth=6
                C(class=p,Purity=0.6769759450171822)
            I(Fidx=1,Score=0.6929774572366698,Split=('y', 'g'))
                    C(class=p,Purity=0.9065040650406504)
                I(Fidx=0,Score=0.4408638638155363,Split=('c',))
                    C(class=p,Purity=1.0)
        I(Fidx=0,Score=0.9604187037495768,Split=('f', 'x'))
                    C(class=p,Purity=0.5418454935622318)
                I(Fidx=0,Score=0.9966965407481536,Split=('x',))
                    C(class=e,Purity=0.5238578680203045)
            I(Fidx=1,Score=0.9903438373831857,Split=('s',))
                    C(class=p,Purity=0.6414048059149723)
                I(Fidx=0,Score=0.9710405141964833,Split=('x',))
                    C(class=p,Purity=0.5539845758354756)
    I(Fidx=1,Score=0.9655127525362506,Split=('f',))
                C(class=e,Purity=0.6753424657534246)
            I(Fidx=0,Score=0.927825296358852,Split=('x',)

In [18]:
pclasses = dt.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print(np.sum(pclasses==Ytest))
print("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

1498
Accuracy =  0.6146901928600739


In [19]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)
feat=np.arange(2)
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)


pclasses=dt.predict(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.60)

In [20]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

dt=DecisionTree(0.95,5,10)
dt.train(Xtrain,Ytrain)

pclasses=dt.predict(Xtest)
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.90)

# Lets Train on all the features and for both the classes....

In [21]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print(" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape )  
print(" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape   )


 Training Data Set Dimensions= (5687, 22) Training True Class labels dimensions (5687,)
 Test Data Set Dimensions= (2437, 22) Test True Class labels dimensions (5687,)


In [22]:
feat = np.arange(22)#[0, 1, 2, 3]
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print(np.sum(pclasses==Ytest))
print("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

2404
Accuracy =  0.9864587607714403


In [23]:
print (dt)

---------------------------------------------------
 A Decision Tree With Depth=2
    C(class=e,Purity=0.9713155291790306)
I(Fidx=4,Score=0.10013167831390946,Split=('p', 's', 'm', 'c', 'f', 'y'))
    C(class=p,Purity=1.0)
---------------------------------------------------


What can you conclude ?
====================
Please write your observation....



# Cross-Validation

Until now we have been splitting the dataset into a training and test set rather randomly and were reporting a rather artifical performance. Now we are going to test our system exhaustively by making use of k-fold [cross validation](http://en.wikipedia.org/wiki/Cross-validation_%28statistics%29). 

Now go and tune your hyper-parameters (purity, exthreshold) to opitmize the performance.

In [24]:
# Now lets cross validate for best paramters, and test the result...
# We will be training four different models on four different partitions of data set and 
# then will be reporting the mean accuracy of the four classifiers.

nfolds = 4 # lets use four folds..
folds = t.generate_folds(X,Y,nfolds)
features = np.arange(22)#[0, 1, 2, 3] # features to use for our system
#now lets train and test on these folds...
totacc=[]
for k in range(nfolds):
    dt = DecisionTree(0.95,5,7)
    dt.train(folds[k][0][:,features],folds[k][1])
    pclasses=dt.predict(folds[k][2][:,features])
    acc=np.sum(pclasses==folds[k][3])/float(folds[k][3].shape[0])
    print(dt)
    print("[Info] Fold {} Accuracy = {}".format(k+1, acc))
    totacc.append(acc)

print(totacc, '\n Mean Accuracy =', np.mean(totacc))

Generating CV data for 2 classes
---------------------------------------------------
 A Decision Tree With Depth=2
    C(class=e,Purity=0.971375807940905)
I(Fidx=4,Score=0.0999520040429716,Split=('p', 's', 'm', 'c', 'f', 'y'))
    C(class=p,Purity=1.0)
---------------------------------------------------
[Info] Fold 1 Accuracy = 0.9867060561299852
---------------------------------------------------
 A Decision Tree With Depth=2
    C(class=e,Purity=0.9704797047970479)
I(Fidx=4,Score=0.10246518181168153,Split=('p', 's', 'm', 'c', 'f', 'y'))
    C(class=p,Purity=1.0)
---------------------------------------------------
[Info] Fold 2 Accuracy = 0.9881831610044313
---------------------------------------------------
 A Decision Tree With Depth=2
    C(class=e,Purity=0.9722735674676525)
I(Fidx=4,Score=0.09741656409029105,Split=('p', 's', 'm', 'c', 'f', 'y'))
    C(class=p,Purity=1.0)
---------------------------------------------------
[Info] Fold 3 Accuracy = 0.9852289512555391
---------------

In [25]:
# Now lets cross validate for best paramters, and test the result...
# We will be training four different models on four different partitions of data set and 
# then will be reporting the mean accuracy of the four classifiers.

nfolds = 4 # lets use four folds..
folds = t.generate_folds(X,Y,nfolds)
features=[0,1, 2, 3] # features to use for our system
#now lets train and test on these folds...

#Lets perform the grid search...
purity = np.linspace(0.85,0.97,13)
nexamp = np.linspace(5,25,21)  

params = np.zeros((len(purity),len(nexamp)))
                   
for p in range(len(purity)): #13 times
    for n in range(len(nexamp)): #21 times
        totacc=[]
        for k in range(nfolds):
            dt = DecisionTree(purity[p],nexamp[n],7)#purity[p],nexamp[n])
            dt.train(folds[k][0][:,features],folds[k][1])
            pclasses=dt.predict(folds[k][2][:,features])
            acc=np.sum(pclasses==folds[k][3])/float(folds[k][3].shape[0])
            print ("[Info] Fold {} Accuracy = {}".format(k+1, acc))
            totacc.append(acc)
        params[p,n] = np.mean(totacc)
        print(totacc, '\nPurity={}, Nexample-threshold={}, Mean Accuracy ={}'.format(purity[p],nexamp[n], np.mean(totacc)))

Generating CV data for 2 classes
[Info] Fold 1 Accuracy = 0.8783850320039389
[Info] Fold 2 Accuracy = 0.8759231905465288
[Info] Fold 3 Accuracy = 0.8828163466272771
[Info] Fold 4 Accuracy = 0.8877400295420975
[0.8783850320039389, 0.8759231905465288, 0.8828163466272771, 0.8877400295420975] 
Purity=0.85, Nexample-threshold=5.0, Mean Accuracy =0.8812161496799606
[Info] Fold 1 Accuracy = 0.8783850320039389
[Info] Fold 2 Accuracy = 0.8759231905465288
[Info] Fold 3 Accuracy = 0.8828163466272771
[Info] Fold 4 Accuracy = 0.8877400295420975
[0.8783850320039389, 0.8759231905465288, 0.8828163466272771, 0.8877400295420975] 
Purity=0.85, Nexample-threshold=6.0, Mean Accuracy =0.8812161496799606
[Info] Fold 1 Accuracy = 0.8783850320039389
[Info] Fold 2 Accuracy = 0.8759231905465288
[Info] Fold 3 Accuracy = 0.8828163466272771
[Info] Fold 4 Accuracy = 0.8877400295420975
[0.8783850320039389, 0.8759231905465288, 0.8828163466272771, 0.8877400295420975] 
Purity=0.85, Nexample-threshold=7.0, Mean Accuracy 

In [26]:
np.unravel_index(np.argmax(params), params.shape)

(9, 15)